# Atelier 5

---

## Instructions

En se basant sur le fichier sql (ci-dessous) et la figure, vous devez réaliser les étapes suivantes en utilisant les requêtes SQL. </br>

![5_schema_figure.png](5_schema_figure.png)

---

### Partie 0 - Imports et fonctions

In [ ]:
import sqlite3 as sq
import mysql.connector as cnt

def connect_db(host, user, pswd, database = None):
    db = cnt.connect(
        host = host,
        user = user,
        passwd = pswd,
        database = database
    )
    return(db)

def exe(db, request, value = None, verbose = False, name = False):
    cursor = db.cursor(buffered = True)
    
    if value:
        cursor.execute(request, value)

    else :
        cursor.execute(request)

    db.commit()
    
    if verbose:
        result = cursor.fetchall()
        if name:
            result.insert(0, cursor.column_names)
        return(result)

---

### Partie 1 - création de la base et des tables :

#### 1.1/ créer une nouelle base données nommée atelier5

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu')

query = 'CREATE DATABASE IF NOT EXISTS atelier5;'

exe(db, query)

#### 1.2/ Imiter les requêtes du fichier sql shema pour créer les tables et les remplir.

```

CREATE TABLE Pieces (
 Code INTEGER PRIMARY KEY NOT NULL,
 Name TEXT NOT NULL
 );
CREATE TABLE Providers (
 Code VARCHAR(40) 
 PRIMARY KEY NOT NULL,  
 Name TEXT NOT NULL 
 );
CREATE TABLE Provides (
 Piece INTEGER, 
 FOREIGN KEY (Piece) REFERENCES Pieces(Code),
 Provider VARCHAR(40), 
 FOREIGN KEY (Provider) REFERENCES Providers(Code),  
 Price INTEGER NOT NULL,
 PRIMARY KEY(Piece, Provider) 
 );
 
INSERT INTO Providers(Code, Name) VALUES('HAL','Clarke Enterprises');
INSERT INTO Providers(Code, Name) VALUES('RBT','Susan Calvin Corp.');
INSERT INTO Providers(Code, Name) VALUES('TNBC','Skellington Supplies');

INSERT INTO Pieces(Code, Name) VALUES(1,'Sprocket');
INSERT INTO Pieces(Code, Name) VALUES(2,'Screw');
INSERT INTO Pieces(Code, Name) VALUES(3,'Nut');
INSERT INTO Pieces(Code, Name) VALUES(4,'Bolt');

INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'HAL',10);
INSERT INTO Provides(Piece, Provider, Price) VALUES(1,'RBT',15);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'HAL',20);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'RBT',15);
INSERT INTO Provides(Piece, Provider, Price) VALUES(2,'TNBC',14);
INSERT INTO Provides(Piece, Provider, Price) VALUES(3,'RBT',50);
INSERT INTO Provides(Piece, Provider, Price) VALUES(3,'TNBC',45);
INSERT INTO Provides(Piece, Provider, Price) VALUES(4,'HAL',5);
INSERT INTO Provides(Piece, Provider, Price) VALUES(4,'RBT',7);

```

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier5')

queries = ["""
    CREATE TABLE IF NOT EXISTS Pieces (
        Code INTEGER PRIMARY KEY NOT NULL,
        Name TEXT NOT NULL
    )ENGINE = InnoDB;
"""]

queries.append("""
    CREATE TABLE IF NOT EXISTS Providers (
        Code VARCHAR(40) 
        PRIMARY KEY NOT NULL,  
        Name TEXT NOT NULL 
    )ENGINE = InnoDB;
""")

queries.append("""
    CREATE TABLE Provides (
        Piece INTEGER, 
        FOREIGN KEY (Piece) REFERENCES Pieces(Code),
        Provider VARCHAR(40), 
        FOREIGN KEY (Provider) REFERENCES Providers(Code),  
        Price INTEGER NOT NULL,
        PRIMARY KEY(Piece, Provider) 
    )ENGINE = InnoDB;
""")

queries.append("""
    INSERT INTO Providers VALUES
        ('HAL','Clarke Enterprises'),
        ('RBT','Susan Calvin Corp.'),
        ('TNBC','Skellington Supplies');
    """)

queries.append("""
    INSERT INTO Pieces VALUES
        (1,'Sprocket'),
        (2,'Screw'),
        (3,'Nut'),
        (4,'Bolt');
""")

queries.append("""
    INSERT INTO Provides VALUES
        (1,'HAL',10),
        (1,'RBT',15),
        (2,'HAL',20),
        (2,'RBT',15),
        (2,'TNBC',14),
        (3,'RBT',50),
        (3,'TNBC',45),
        (4,'HAL',5),
        (4,'RBT',7);
""")

for query in queries:
    exe(db, query)

---

### Partie 2 - réaliser une sélection de données :

#### 2.1/ Sélectionnez le nom de toutes les pièces.

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier5")

query = """
    SELECT Name
    FROM Pieces;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

#### 2.2/ Sélectionnez les données de tous les fournisseurs(providers).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier5")

query = """
    SELECT *
    FROM Providers;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response}")

#### 2.3/ Obtenir le prix moyen de chaque pièce (ne montrer que le code de la pièce et le prix moyen). Utiliser la fonction AVG et GROUP BY.


In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier5")

query = """
    SELECT Piece, AVG(Price)
    FROM Provides
    GROUP BY Piece;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response}")

#### 2.4/ Obtenir les noms de tous les fournisseurs qui fournissent la pièce 1.

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier5")

query = """
    SELECT Name
    FROM Providers
    INNER JOIN Provides ON Provides.Provider = Providers.Code
    WHERE Piece = 1;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

---

### Partie 3 - Utiliser les jointures :

#### 3.1/ Sélectionner le nom des pièces fournies par le fournisseur avec le code "HAL".

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = "atelier5")

query = """
    SELECT Pieces.Name
    FROM Pieces
    INNER JOIN Provides ON Provides.Piece = Pieces.Code
    WHERE Provider = 'HAL';
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response[0]}")

---

### Partie 4 - réaliser des modifications des tables :

#### 4.1/ Ajouter une entrée dans la base de données pour indiquer que "Skellington Supplies" (code "TNBC") fournira des pignons (code "1") pour 7 cents chacun.

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier5')

query = """
    INSERT INTO Provides VALUES (1, 'TNBC', 7);
"""

exe(db, query)

#### 4.2/ Augmenter tous les prix d'un centime.

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier5')

query = """
    UPDATE Provides
    SET Price = Price + 1;
"""

responses = exe(db, query)

#### 4.3/ Mettre à jour la base de données pour indiquer que "Susan Calvin Corp. (code "RBT") ne fournira pas de boulons (code 4).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier5')

query = """
    DELETE FROM Provides
    WHERE Piece = 4 AND Provider = 'RBT';
"""

responses = exe(db, query)

---

### Partie 5 - BONUS :

#### 5.1/ Pour chaque pièce, trouvez l'offre la plus chère de cette pièce et indiquez le nom de la pièce, le nom du fournisseur et le prix

-- (notez qu'il peut y avoir deux fournisseurs qui fournissent la même pièce au prix le plus élevé).

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu', database = 'atelier5')

query = """
    SELECT
    Pieces.Name,
    Providers.Name,
    MAX(Provides.Price)
    FROM Provides
    JOIN Pieces
    ON Pieces.Code = Provides.Piece
    JOIN Providers
    ON Providers.Code = Provides.Provider
    GROUP BY Pieces.Code;
"""

responses = exe(db, query, verbose = True)

for response in responses:
    print(f"{response}")